In [22]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np
import random

In [23]:
def create_superpixel(height, weight, img):
    return SuperPixels(height, weight, img[height,weight][0], img[height, weight][1], img[height,weight][2])

In [24]:
def init_center_of_claster(S, img, img_height, img_weight, clusters):
    height = S // 2
    weight = S // 2
    while height < img_height:
        while weight < img_weight:
#             Добавил рандом для пункта 1
            rnd_h = random.randint(height, img_height - 1)
            rnd_w = random.randint(weight, img_weight - 1)
            clusters.append(create_superpixel(rnd_h, rnd_w, img))
            weight += S
        weight = S // 2
        height += S
    return clusters

In [25]:
def calculate_gradient(height, weight, img, img_weight, img_height):
    if weight + 1 >= img_weight:
        weight = img_weight - 2
    if height + 1 >= img_height:
        height = img_height - 2
    grad = img[weight + 1, height + 1][0] - img[weight, height][0] + img[weight + 1, height + 1][1] - img[weight, height][1] + img[weight + 1, height + 1][2] - img[weight, height][2]
    return grad

In [26]:
def reassign_cluster_center_acc_to_grad(clusters, img):
    for c in clusters:
        cluster_gradient = calculate_gradient(c.height, c.weight, img, img_weight, img_height)
        for dh in range(-1, 2):
            for dw in range(-1, 2):
                Height = c.height + dh
                Weight = c.weight + dw
                new_gradient = calculate_gradient(Height, Weight, img, img_weight, img_height)
                if new_gradient < cluster_gradient:
                    c.update(Height, Weight, img[Height, Weight][0], img[Height, Weight][1], img[Height, Weight][2])
                    c_gradient = new_gradient

In [27]:
# metric method
def assign_pixels_to_cluster_Euclidean_distance(clusters, S, img, img_height, img_weight, tag, dis):
    for c in clusters:
        for height in range(c.height - 2 * S, c.height + 2 * S):
            if height < 0 or height >= img_height: continue
            for weight in range(c.weight - 2 * S, c.weight + 2 * S):
                if weight < 0 or weight >= img_weight: continue
                l, a, b = img[height, weight]
                Dc = math.sqrt(math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2))
                Ds = math.sqrt(math.pow(height - c.height, 2) + math.pow(weight - c.weight, 2))
                D = math.sqrt(math.pow(Dc / m, 2) + math.pow(Ds /S, 2))
                if D < dis[height, weight]:
                    if (height, weight) not in tag:
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    else:
                        tag[(height, weight)].pixels.remove((height, weight))
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    dis[height, weight] = D

In [28]:
# metric method
def assign_pixels_to_cluster_Euclidean_distance_squared(clusters, S, img, img_height, img_weight, tag, dis):
    for c in clusters:
        for height in range(c.height - 2 * S, c.height + 2 * S):
            if height < 0 or height >= img_height: continue
            for weight in range(c.weight - 2 * S, c.weight + 2 * S):
                if weight < 0 or weight >= img_weight: continue
                l, a, b = img[height, weight]
                Dc = math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2)
                Ds = math.pow(height - c.height, 2) + math.pow(weight - c.weight, 2)
                D = math.pow(Dc / m, 2) + math.pow(Ds /S, 2)
                if D < dis[height, weight]:
                    if (height, weight) not in tag:
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    else:
                        tag[(height, weight)].pixels.remove((height, weight))
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    dis[height, weight] = D

In [42]:
# metric method
def assign_pixels_to_cluster_Chebyshev_dist(clusters, S, img, img_height, img_weight, tag, dis):
    for c in clusters:
        for height in range(c.height - 2 * S, c.height + 2 * S):
            if height < 0 or height >= img_height: continue
            for weight in range(c.weight - 2 * S, c.weight + 2 * S):
                if weight < 0 or weight >= img_weight: continue
                l, a, b = img[height, weight]
                Dc = max(abs(l - c.l), abs(a - c.a), abs(b - c.b))
                Ds = max(abs(height - c.height), abs(weight - c.weight))
                D = (Dc / m) + (Ds / S)
                if D < dis[height, weight]:
                    if (height, weight) not in tag:
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    else:
                        tag[(height, weight)].pixels.remove((height, weight))
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    dis[height, weight] = D

In [46]:
# metric method
def assign_pixels_to_cluster_Manh_dist(clusters, S, img, img_height, img_weight, tag, dis):
    for c in clusters:
        for height in range(c.height - 2 * S, c.height + 2 * S):
            if height < 0 or height >= img_height: continue
            for weight in range(c.weight - 2 * S, c.weight + 2 * S):
                if weight < 0 or weight >= img_weight: continue
                l, a, b = img[height, weight]
                Dc = abs(l - c.l) + abs(a - c.a) + abs(b - c.b)
                Ds = abs(height - c.height) + abs(weight - c.weight)
                D = (Dc / m) + (Ds / S)
                if D < dis[height, weight]:
                    if (height, weight) not in tag:
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    else:
                        tag[(height, weight)].pixels.remove((height, weight))
                        tag[(height, weight)] = c
                        c.pixels.append((height, weight))
                    dis[height, weight] = D

In [31]:
def update_cluster_mean(clusters):
    for c in clusters:
        sum_height = sum_weight = number = 0
        for p in c.pixels:
            sum_height += p[0]
            sum_weight += p[1]
            number += 1
            Height = sum_height // number
            Weight = sum_weight // number
            c.update(Height, Weight, img[Height, Weight][0], img[Height, Weight][1], img[Height, Weight][2])

In [32]:
def avg_color_cluster(img,name,clusters):
    image = np.copy(img)
    for c in clusters:
        for p in c.pixels:
            image[p[0],p[1]][0] = c.l
            image[p[0],p[1]][1] = c.a
            image[p[0],p[1]][2] = c.b
        # To change the color of cluster center to Black
        image[c.height, c.weight][0] = 0
        image[c.height, c.weight][1] = 0
        image[c.height, c.weight][2] = 0
    lab2rgb(name, image)

In [33]:
def slic(S, img, img_height, img_weight, clusters, tag, dis, method):
    clusters = init_center_of_claster(S, img, img_height, img_weight, clusters)
    reassign_cluster_center_acc_to_grad(clusters, img)
    for i in range(10):
        if (method == 1):
            assign_pixels_to_cluster_Euclidean_distance(clusters, S, img, img_height, img_weight, tag, dis)
        elif (method == 2):
            assign_pixels_to_cluster_Euclidean_distance_squared(clusters, S, img, img_height, img_weight, tag, dis)
        elif (method == 3):
            assign_pixels_to_cluster_Chebyshev_dist(clusters, S, img, img_height, img_weight, tag, dis)
        elif (method == 4):
            assign_pixels_to_cluster_Manh_dist(clusters, S, img, img_height, img_weight, tag, dis)
        else:
            print('неправильно введен метод')
            break
        update_cluster_mean(clusters)
        if i == 9 : 
            name = 'out_m{m}_k{k}.png'.format(loop = i, m = m, k = k)
            avg_color_cluster(img, name, clusters)
    return clusters

In [34]:
def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    io.imsave(path, rgb_arr)

In [35]:
class SuperPixels(object):

    def __init__(self, height, weight, l = 0, a = 0, b = 0):
        self.update(height, weight, l, a, b)
        self.pixels = []

    def update(self, height, weight, l, a, b):
        self.height = height
        self.weight = weight
        self.l = l
        self.a = a
        self.b = b

In [47]:
rgb = io.imread('test.jpg', plugin='matplotlib')
print(rgb.shape)

img = resize(rgb, (400,400), anti_aliasing=True)
print(img.shape)

# convert RGB to LAB
img = color.rgb2lab(img)

k = 100 # number of superpixel used to segment the input img
m = 20    

img_height = img.shape[0] 
img_weight = img.shape[1] 

N = img_height * img_weight # number of pixels in the input image
# for roughly equally sized superpixels there would be a superpixel centre at every grid interval S
S = int(math.sqrt(N /k)) 

clusters = []
tag = {}
dis = np.full((img_height, img_weight), np.inf)

# №3 Выбор метрики
print('Введите номер метрики от 1 до 4')
method = input()
method = int(method)

if (method == 1 or method == 2 or method == 3 or method == 4):
    cluster = slic(S, img, img_height, img_weight, clusters, tag, dis, method)
else:
    print("неправильно введен номер метрики")

(320, 240, 3)
(400, 400, 3)
Введите номер метрики от 1 до 4
4


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
